# Import / Config

In [5]:
import pandas as pd
from sqlalchemy import create_engine, text
from neo4j import GraphDatabase
import cred as c

## Postgres

In [2]:
engine = create_engine(
    f'postgresql://{c.pg_userid}:{c.pg_password}@{c.pg_host}/{c.pg_db}', 
    connect_args = {'options': '-c search_path=usozmed,public', 'keepalives_idle': 120},
    pool_size=1, 
    max_overflow=0,
    execution_options={ 'isolation_level': 'AUTOCOMMIT' }
)

In [3]:
# engine.dispose()

## Person

In [8]:
sql = "select id, lastname from person"
with engine.connect() as con:
    df = pd.read_sql_query(text(sql), con)

cypher = "CREATE (p:Person {id: $id, lastname: $lastname})"
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        for r in df.itertuples(index=False):
            session.run(cypher, id=r.id, lastname=r.lastname)